In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dt_apriltags import Detector



def detect_lines(img, threshold1 = 50, threshold2 = 150, apertureSize = 3,  minLineLength = 100, maxLineGap = 10):
    img = cv2.imread(img)
    img = cv2.resize(img, (1100, 650))
    height = img.shape[:2]
    img = img[:height//2, :]  # This slices the top half of the image

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    edges = cv2.Canny(gray, threshold1, threshold2, apertureSize=apertureSize) # detect edges
    lines = cv2.HoughLinesP(
                    edges,
                    1,
                    np.pi/180,
                    100,
                    minLineLength=minLineLength,
                    maxLineGap=maxLineGap,
            ) # detect lines

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    print(len(lines))
    plt.imshow(img)

    print(img.shape)



def draw_lines(img, lines, color = (0, 255, 0)):
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(img, (x1, y1), (x2, y2), color, 2)

    plt.imshow(img)

def get_slopes_intercepts(lines):
    slopes = []
    intercepts = []
    for line in lines:
        
        x1 = line[0][0]
        y1 = line[0][1]
        x2 = line[0][2]
        y2 = line[0][3]

        slope = (y2-y1)/(x2-x1)
        intercept = -slope*x2
        slopes.append(slope)
        intercepts.append(intercept)
        
    return slopes, intercepts

def detect_lanes(lines):
    # Get the slopes and intercepts of the lines using the get_slopes_intercepts function.
    # Check if a pair of lines is a lane.
    # Return the list of lanes. Each lane should be a list of two lines.
    outputs = get_slopes_intercepts(lines)

    slopes, intercepts = outputs
    pairs = []
    for i in range(len(outputs)):

        tempPairs = []
        EPSILON = 2
        OMICRON = 200
        for j in range(len(outputs)):
            
            if j == i:
                continue

            # Checks for pairs that are within EPSILON
            if slopes[i] >= slopes[j] - EPSILON and slopes[i] <= slopes[j] + EPSILON:

                # Checks for pairs whoose intercepts are within OMNICRON
                if intercepts[i] >= intercepts[j] - OMICRON and intercepts[i] <= intercepts[j] + OMICRON:
                    if len(tempPairs < 2):
                        tempPairs.append(outputs[i])
                        tempPairs.append(outputs[j])

        pairs.append(tempPairs)

    return pairs


